In [16]:
import os
from pprint import pprint

In [2]:
%ls

'HEA_IE extraction evaluation(RC TR version).ipynb'   schema_0shot_simple/
 langchain_0shot_complex/                             schema_1shot_complex/
 langchain_0shot_simple/                              schema_1shot_simple/
 schema_0shot_complex/


In [3]:
data_path = '../RMPEA Dataset filtered Aug 30 23.csv'
assert os.path.exists(data_path)

In [4]:
extraction_dir = 'schema_0shot_simple/'

In [5]:
%pip install pymatgen

  Using cached numpy-1.23.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorboard 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
detoxify 0.5.1 requires transformers==4.22.1, but you have transformers 4.38.1 which is incompatible.
decontext 0.1.7 requires openai==0.27.7, but you have openai 1.17.1 which is incompatible.
decontext 0.1.7 requires tiktoken==0.4.0, but you have tiktoken 0.6.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [6]:
# %pip install numpy==1.24.4

# Read paper dataset

In [7]:
import pandas as pd

In [8]:
paper_df = pd.read_csv(data_path)
display(paper_df.shape)
display(paper_df.head(5))

(971, 23)

,IDENTIFIER: Reference ID,FORMULA,PROPERTY: Microstructure,PROPERTY: Processing method,PROPERTY: BCC/FCC/other,PROPERTY: grain size ($\mu$m),PROPERTY: Exp. Density (g/cm$^3$),PROPERTY: Computed Density (g/cm$^3$),PROPERTY: HV,PROPERTY: Type of test,...,PROPERTY: Elongation (%),PROPERTY: Elongation plastic (%),PROPERTY: Exp. Young modulus (GPa),PROPERTY: Computed Young modulus (GPa),PROPERTY: O content (wppm),PROPERTY: N content (wppm),PROPERTY: C content (wppm),REFERENCE: doi,REFERENCE: year,REFERENCE: title
0,7,Al0.025 Cr0.2 Mo0.2 Nb0.2 Ta0.025 Ti0.2 V0.05 ...,BCC+Laves,CAST,other,NaN,7.44,7.5,NaN,C,...,NaN,NaN,NaN,185.0,NaN,NaN,NaN,10.1016/j.jallcom.2020.155700,2020.0,Combinatorial development of the low-density h...
1,7,Al0.025 Cr0.2 Mo0.2 Nb0.2 Ta0.05 Ti0.2 V0.025 ...,BCC+Laves,CAST,other,NaN,7.77,7.8,NaN,C,...,NaN,NaN,NaN,187.0,NaN,NaN,NaN,10.1016/j.jallcom.2020.155700,2020.0,Combinatorial development of the low-density h...
2,7,Al0.025 Cr0.2 Mo0.2 Nb0.2 Ta0.075 Ti0.2 Zr0.1,BCC+Laves,CAST,other,NaN,8.06,8.1,NaN,C,...,NaN,NaN,NaN,188.0,NaN,NaN,NaN,10.1016/j.jallcom.2020.155700,2020.0,Combinatorial development of the low-density h...
3,7,Al0.025 Cr0.2 Mo0.2 Nb0.2 Ti0.2 V0.075 Zr0.1,BCC+Laves,CAST,other,NaN,7.19,7.3,NaN,C,...,NaN,NaN,NaN,184.0,NaN,NaN,NaN,10.1016/j.jallcom.2020.155700,2020.0,Combinatorial development of the low-density h...
4,1,Al0.038 Mo0.192 Nb0.192 Ta0.192 Ti0.192 V0.192,BCC,CAST,BCC,NaN,NaN,9.0,474.0,C,...,16.0,NaN,NaN,169.0,NaN,NaN,NaN,10.1016/j.msea.2020.139275,2020.0,Effects of Al addition on the microstructures ...


## Analyzing unique keys

In [9]:
gb = paper_df.groupby(['REFERENCE: doi','FORMULA','PROPERTY: YS (MPa)'])

In [10]:
len(gb.groups)

883

In [11]:
for group in gb.groups:
  print(gb.indices)
  break

{('10.1002/adem.202100765', 'Mo0.25 Re0.25 Ta0.25 W0.25', 172.0): array([823]), ('10.1002/adem.202100765', 'Mo0.25 Re0.25 Ta0.25 W0.25', 1075.0): array([822]), ('10.1002/adma.201701678', 'Hf0.25 Ta0.25 Ti0.25 Zr0.25', 1356.0): array([622]), ('10.1002/adma.201701678', 'Hf0.278 Ta0.167 Ti0.278 Zr0.278', 750.0): array([627]), ('10.1002/adma.201701678', 'Hf0.286 Ta0.143 Ti0.286 Zr0.286', 687.0): array([628]), ('10.1002/adma.201701678', 'Hf0.294 Ta0.118 Ti0.294 Zr0.294', 354.0): array([631]), ('10.1007/s00339-019-2484-1', 'Hf0.2 Sc0.2 Ti0.2 Y0.2 Zr0.2', 793.0): array([569]), ('10.1007/s00339-019-2484-1', 'Hf0.25 Sc0.25 Ti0.25 Zr0.25', 1001.0): array([621]), ('10.1007/s00339-019-2484-1', 'Hf0.25 Ti0.25 Y0.25 Zr0.25', 554.0): array([625]), ('10.1007/s00339-019-2484-1', 'Hf0.333 Ti0.333 Zr0.333', 773.0): array([639]), ('10.1007/s10853-012-6260-2', 'Hf0.2 Nb0.2 Ta0.2 Ti0.2 Zr0.2', 92.0): array([470]), ('10.1007/s10853-012-6260-2', 'Hf0.2 Nb0.2 Ta0.2 Ti0.2 Zr0.2', 295.0): array([469]), ('10.1007

In [12]:
v_groups = {group:indices for group, indices in gb.indices.items() if len(indices) > 1}
v_groups

{('10.1016/j.actamat.2019.06.006',
  'Al0.25 Nb0.25 Ta0.25 Ti0.25',
  740.0): array([249, 250]),
 ('10.1016/j.actamat.2019.06.006',
  'Cr0.25 Mo0.25 Ta0.25 Ti0.25',
  1162.0): array([330, 331]),
 ('10.1016/j.msea.2017.10.073',
  'Hf0.15 Nb0.2 Ta0.1 Ti0.3 Zr0.25',
  1020.0): array([402, 409]),
 ('10.1016/j.msea.2017.10.073',
  'Hf0.15 Nb0.2 Ta0.1 Ti0.3 Zr0.25',
  1370.0): array([396, 400])}

In [13]:
pd.set_option("display.max_columns", None)
paper_df.iloc[[396, 400]]

,IDENTIFIER: Reference ID,FORMULA,PROPERTY: Microstructure,PROPERTY: Processing method,PROPERTY: BCC/FCC/other,PROPERTY: grain size ($\mu$m),PROPERTY: Exp. Density (g/cm$^3$),PROPERTY: Computed Density (g/cm$^3$),PROPERTY: HV,PROPERTY: Type of test,PROPERTY: Test temperature ($^\circ$C),PROPERTY: YS (MPa),PROPERTY: UTS (MPa),PROPERTY: Elongation (%),PROPERTY: Elongation plastic (%),PROPERTY: Exp. Young modulus (GPa),PROPERTY: Computed Young modulus (GPa),PROPERTY: O content (wppm),PROPERTY: N content (wppm),PROPERTY: C content (wppm),REFERENCE: doi,REFERENCE: year,REFERENCE: title
396,74,Hf0.15 Nb0.2 Ta0.1 Ti0.3 Zr0.25,BCC,WROUGHT,BCC,NaN,NaN,8.4,NaN,C,-153.0,1370.0,NaN,NaN,NaN,NaN,103.0,NaN,NaN,NaN,10.1016/j.msea.2017.10.073,2018.0,Mechanical properties and thermally activated ...
400,74,Hf0.15 Nb0.2 Ta0.1 Ti0.3 Zr0.25,BCC,WROUGHT,BCC,NaN,NaN,8.4,NaN,C,-103.0,1370.0,NaN,NaN,NaN,NaN,103.0,NaN,NaN,NaN,10.1016/j.msea.2017.10.073,2018.0,Mechanical properties and thermally activated ...


# Read extraction jsons

In [14]:
os.listdir(extraction_dir)


['10.1007.s40195-019-00935-x_extracted_info.json',
 '10.1016.j.jallcom.2018.10.230_extracted_info.json',
 '10.1016.j.jallcom.2018.11.111_extracted_info.json',
 '10.1007.s10853-012-6260-2_extracted_info.json',
 '10.1016.j.msea.2020.139774_extracted_info.json',
 '10.1016.j.matlet.2016.03.133_extracted_info.json',
 '10.1016.j.jmst.2020.05.038_extracted_info.json',
 '10.1016.j.ijrmhm.2021.105664_extracted_info.json',
 '10.1016.j.ijrmhm.2019.02.009_extracted_info.json',
 'failed_DOIs.json',
 '10.1016.j.intermet.2017.01.007_extracted_info.json',
 '10.1038.s41563-020-0750-4_extracted_info.json',
 '10.1016.j.matlet.2020.127372_extracted_info.json',
 '10.1016.j.msea.2017.08.019_extracted_info.json',
 '10.1016.j.msec.2016.12.057_extracted_info.json',
 '10.1016.j.jallcom.2021.159190_extracted_info.json',
 '10.1016.j.matdes.2015.05.019_extracted_info.json',
 '10.1007.s11837-014-1066-0_extracted_info.json',
 '10.1016.j.matdes.2017.11.033_extracted_info.json',
 '10.1016.j.jallcom.2015.11.050_extract

In [61]:
import json
extraction_dfs = []
extraction_dois = set()
for extraction_fn in os.listdir(extraction_dir):
  if extraction_fn == 'failed_DOIs.json':
    continue
  print(extraction_fn)
  extraction_path = os.path.join(extraction_dir, extraction_fn)
  with open(extraction_path) as f:
    obj = json.load(f)

  doi = extraction_fn.replace('_extracted_info.json','').replace(']','')
  extraction_dois.add(doi)
  
  try:
    records = [obj['extracted_info'] if type(obj['extracted_info']) == dict else entry for entry in obj['extracted_info']]
  except:
    print(extraction_fn)
    print(obj)
    continue
  
  df = pd.DataFrame(records)
  df['doi'] = doi
  
  print(df.head(1))
  extraction_dfs.append(df)
  
  
  
  ''' commenting out everything below this line for time being
  
  # if 'extracted_info' in obj['extracted_info'].keys():
  #   print('a list of dict with key')
  # else:
  #   print(obj['extracted_info'])

  # Slightly redundant schema
  pprint(obj['extracted_info'])
  records = [entry['extracted_info'] if 'extracted_info' in obj['extracted_info'].keys() else entry for entry in obj['extracted_info']]
  pprint(records)
  df = pd.DataFrame(records)
  df['doi'] = doi

  display(df.shape)
  display(df.head(1))
  extraction_dfs.append(df)
  print('\n','*'*50,'\n')
  '''

print('Extraction DFs:',len(extraction_dfs))

10.1007.s40195-019-00935-x_extracted_info.json
  high entropy alloy formula  yield strength                         doi
0                  MoNbFeCrV            2663  10.1007.s40195-019-00935-x
10.1016.j.jallcom.2018.10.230_extracted_info.json
  high entropy alloy formula      grain size experimental density  hardness  \
0                    NbTaTiV  No information       No information     510.9   

   test temperature  yield strength ultimate tensile strength      elongation  \
0              1000             437            No information  No information   

   elongation plastic                            doi  
0                  40  10.1016.j.jallcom.2018.10.230  
10.1016.j.jallcom.2018.11.111_extracted_info.json
  high entropy alloy formula      grain size experimental density  \
0      MoNbRe 0.5 W(TaC) 0.5  No information       No information   

         hardness test temperature  yield strength  ultimate tensile strength  \
0  No information   No information          1202.0     

In [62]:
extraction_dfs

[  high entropy alloy formula  yield strength                         doi
 0                  MoNbFeCrV            2663  10.1007.s40195-019-00935-x
 1                  MoNbFeCrV            2663  10.1007.s40195-019-00935-x,
   high entropy alloy formula      grain size experimental density  hardness  \
 0                    NbTaTiV  No information       No information     510.9   
 1                    NbTaTiV  No information       No information     510.9   
 2                    NbTaTiV  No information       No information     510.9   
 3                    NbTaTiV  No information       No information     510.9   
 4                    NbTaTiV  No information       No information     510.9   
 5                    NbTaTiV  No information       No information     510.9   
 6                    NbTaTiV  No information       No information     510.9   
 7                    NbTaTiV  No information       No information     510.9   
 8                    NbTaTiV  No information       No in

In [64]:
colname_map = {'FORMULA':'high entropy alloy formula',
               'PROPERTY: grain size ($\mu$m)':'grain size',
               'PROPERTY: YS (MPa)':'yield strength',
               'PROPERTY: Exp. Density (g/cm$^3$)':'experimental density',
               'PROPERTY: HV':'hardness',
               'PROPERTY: Test temperature ($^\circ$C)':'test temperature',
               'PROPERTY: UTS (MPa)':'ultimate tensile strength',
               'PROPERTY: Elongation (%)':'elongation',
               'PROPERTY: Elongation plastic (%)':'elongation plastic',
               'REFERENCE: doi':'doi'
               }

In [66]:
extraction_df = pd.concat(extraction_dfs, axis=0).reset_index(drop=True)
# extraction_df.drop(columns=['ultimate tensible strength'], inplace=True)
display(extraction_df.shape)
columns_to_drop = [col for col in extraction_df.columns if col not in colname_map.values()]
extraction_df.drop(columns=columns_to_drop, inplace=True)
display(extraction_df)

(860, 91)

,high entropy alloy formula,yield strength,doi,grain size,experimental density,hardness,test temperature,ultimate tensile strength,elongation,elongation plastic
0,MoNbFeCrV,2663,10.1007.s40195-019-00935-x,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MoNbFeCrV,2663,10.1007.s40195-019-00935-x,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NbTaTiV,437,10.1016.j.jallcom.2018.10.230,No information,No information,510.9,1000,No information,No information,40
3,NbTaTiV,437,10.1016.j.jallcom.2018.10.230,No information,No information,510.9,1000,No information,No information,40
4,NbTaTiV,437,10.1016.j.jallcom.2018.10.230,No information,No information,510.9,1000,No information,No information,40
...,...,...,...,...,...,...,...,...,...,...
855,NaN,NaN,10.1016.j.intermet.2015.10.011,NaN,NaN,NaN,NaN,NaN,NaN,NaN
856,NaN,NaN,10.1016.j.intermet.2015.10.011,NaN,NaN,NaN,NaN,NaN,NaN,NaN
857,NaN,NaN,10.1016.j.intermet.2015.10.011,NaN,NaN,NaN,NaN,NaN,NaN,NaN
858,NaN,NaN,10.1016.j.intermet.2015.10.011,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
import numpy as np
extraction_df.replace(to_replace='No information', value=np.nan, inplace=True)
display(extraction_df.head(3))

,high entropy alloy formula,yield strength,doi,grain size,experimental density,hardness,test temperature,ultimate tensile strength,elongation,elongation plastic
0,MoNbFeCrV,2663,10.1007.s40195-019-00935-x,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MoNbFeCrV,2663,10.1007.s40195-019-00935-x,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NbTaTiV,437,10.1016.j.jallcom.2018.10.230,NaN,NaN,510.9,1000,NaN,NaN,40


# Aligning extracted and ground-truth papers

In [68]:
paper_df.iloc[0]

IDENTIFIER: Reference ID                                                                  7
FORMULA                                   Al0.025 Cr0.2 Mo0.2 Nb0.2 Ta0.025 Ti0.2 V0.05 ...
PROPERTY: Microstructure                                                          BCC+Laves
PROPERTY: Processing method                                                            CAST
PROPERTY: BCC/FCC/other                                                               other
PROPERTY: grain size ($\mu$m)                                                           NaN
PROPERTY: Exp. Density (g/cm$^3$)                                                      7.44
PROPERTY: Computed Density (g/cm$^3$)                                                   7.5
PROPERTY: HV                                                                            NaN
PROPERTY: Type of test                                                                    C
PROPERTY: Test temperature ($^\circ$C)                                          

In [69]:
extraction_df.iloc[0]

high entropy alloy formula                     MoNbFeCrV
yield strength                                      2663
doi                           10.1007.s40195-019-00935-x
grain size                                           NaN
experimental density                                 NaN
hardness                                             NaN
test temperature                                     NaN
ultimate tensile strength                            NaN
elongation                                           NaN
elongation plastic                                   NaN
Name: 0, dtype: object

## Hardcoded column name mapping

In [70]:
for col in extraction_df.columns:
  assert col in colname_map.values(), print(col)

for col in colname_map.values():
  assert col in extraction_df.columns

for col in colname_map.keys():
  assert col in paper_df.columns, print(col)

inverted_colname_map = {value:key for key,value in colname_map.items()}

In [71]:
extraction_df.columns

Index(['high entropy alloy formula', 'yield strength', 'doi', 'grain size',
       'experimental density', 'hardness', 'test temperature',
       'ultimate tensile strength', 'elongation', 'elongation plastic'],
      dtype='object')

## Matching up DOIs

In [72]:
paper_df['REFERENCE: doi']

0      10.1016/j.jallcom.2020.155700
1      10.1016/j.jallcom.2020.155700
2      10.1016/j.jallcom.2020.155700
3      10.1016/j.jallcom.2020.155700
4         10.1016/j.msea.2020.139275
                   ...              
966       10.1016/j.msea.2021.141168
967       10.1016/j.msea.2021.141168
968                10.3390/e21020114
969       10.1016/j.msea.2003.12.011
970       10.1016/j.msea.2003.12.011
Name: REFERENCE: doi, Length: 971, dtype: object

In [73]:
extraction_df['doi']

0          10.1007.s40195-019-00935-x
1          10.1007.s40195-019-00935-x
2       10.1016.j.jallcom.2018.10.230
3       10.1016.j.jallcom.2018.10.230
4       10.1016.j.jallcom.2018.10.230
                    ...              
855    10.1016.j.intermet.2015.10.011
856    10.1016.j.intermet.2015.10.011
857    10.1016.j.intermet.2015.10.011
858    10.1016.j.intermet.2015.10.011
859    10.1016.j.intermet.2015.10.011
Name: doi, Length: 860, dtype: object

In [74]:
#Should be 20 or so uniques, unless there were papers from which satanu couldn't extract anything
# extraction_dois = extraction_df['doi'].unique()
len(extraction_dois)

126

In [75]:
# Two possible ways to find the set of ground-truth records:
# 1: match on DOIs from the extraction dataset
# 2: Grab the top 20 alphabetical paper titles, which is supposedly what Satanu did

# Alphabetical approach not working because the extracted records appear to include the top 21st, 22nd, and 23rd titles,
# but are missing 4 titles that are above them (see below for analysis)

# DOI matching mostly works, but it is unable to find the 20th paper attempted by Satanu, which means that the performance
# will be slightly overestimated, because it will not account for the paper for which, presumably, Satanu's extraction
# wasn't able to find anything

paper_df['processed_doi'] =  paper_df['REFERENCE: doi'].apply(lambda doi:doi.replace('/','.'))

# filtered_paper_df = paper_df[paper_df['processed_doi'].isin(extraction_dois)]

# # So this isn't ideal. Should be 20. Need to get list of 20 dois from Satanu, but will continue onward for now.
# len(filtered_paper_df['REFERENCE: doi'].unique())
first_20_papers = sorted(paper_df['REFERENCE: title'][paper_df['REFERENCE: title'].notnull()].unique())[0:20]
alphabetical_filtered_paper_df = paper_df[paper_df['REFERENCE: title'].isin(first_20_papers)]

doi_match_filtered_paper_df = paper_df[paper_df['processed_doi'].isin(extraction_dois)]

filtered_paper_df =doi_match_filtered_paper_df #Using the DOI matching for now til I resolve things with Satanu

print(filtered_paper_df.shape)
print('Filtered titles:',len(filtered_paper_df['REFERENCE: title'].unique())) # should be 20
print('Filtered DOIs:',len(filtered_paper_df['REFERENCE: doi'].unique())) # should be 20


filtered_paper_dois = filtered_paper_df['REFERENCE: doi'].apply(lambda doi:doi.replace('/','.')).unique()

overlap = set(filtered_paper_dois).intersection(set(extraction_dois))
print('Overlapping DOIs:',len(overlap)) #should be size of smaller set
assert len(overlap) == min(len(filtered_paper_dois), len(extraction_dois))

(946, 24)
Filtered titles: 126
Filtered DOIs: 126
Overlapping DOIs: 126


In [76]:
# Analyze the non-overlap between the alphabetical ground truth papers and extracted papers
processed_filtered_dois = set(alphabetical_filtered_paper_df['processed_doi'].unique())

print('Extracted but not included in the alphabetical filtered paper DF:')
extracted_not_filtered_dois = (set(extraction_dois).difference(processed_filtered_dois))
print(extracted_not_filtered_dois)
display(paper_df[paper_df['processed_doi'].isin(extracted_not_filtered_dois)]['REFERENCE: title'].unique())
# So Satanu's extracted results includes some papers which shouldn't be in the alphabetical top 20


print('\nPresent in alphabetical filtered paper df but not extracted')
filtered_not_extracted = (set(processed_filtered_dois).difference(extraction_dois))
print(filtered_not_extracted)
display(paper_df[paper_df['processed_doi'].isin(filtered_not_extracted)]['REFERENCE: title'].unique())
# And it excludes some papers which should be in the top 20, alphabetically


Extracted but not included in the alphabetical filtered paper DF:
{'10.1016.j.jmst.2020.07.012', '10.1016.j.jallcom.2015.11.091', '10.3390.e21010015', '10.1016.j.ijrmhm.2020.105370', '10.1016.j.matlet.2019.03.075', '10.1016.j.ijrmhm.2019.105132', '10.1016.j.ijrmhm.2021.105664', '10.1016.j.scriptamat.2020.09.027', '10.1016.j.matlet.2021.129753', '10.1016.j.intermet.2011.01.004', '10.1016.j.scriptamat.2021.114225', '10.1016.j.matlet.2021.129428', '10.1016.j.msea.2015.08.024', '10.1007.s11837-020-04557-y', '10.1016.j.msea.2017.12.004', '10.1179.1743284715Y.0000000032', '10.1016.j.msea.2003.12.011', '10.1016.j.msea.2021.140798', '10.1016.j.msea.2017.12.021', '10.1016.j.matlet.2016.03.133', '10.1007.s40195-019-00935-x', '10.1016.j.jmst.2020.05.038', '10.1016.j.actamat.2017.09.035', '10.1002.adma.201701678', '10.1016.j.ijrmhm.2019.02.009', '10.1016.j.matchar.2021.111287', '10.1016.j.actamat.2019.06.006', '10.1016.j.vacuum.2020.109614', '10.1016.j.ijrmhm.2020.105467', '10.1016.j.intermet.2020

array(['Effects of Al addition on the microstructures and properties of MoNbTaTiV refractory high entropy alloy',
       'Lightweight Zr1.2V0.8NbTi Al  high-entropy alloys with high tensile strength and ductility',
       'The mechanical and oxidation properties of novel B2-ordered Ti2ZrHf0.5VNb0.5Alx refractory high-entropy alloys',
       'Microstructure and Mechanical Properties of VTaTiMoAlsubx/sub Refractory High Entropy Alloys',
       'Microstructures and mechanical properties of Ta‚ÄìNb‚ÄìZr‚ÄìTi‚ÄìAl refractory high entropy alloys with varying Ta/Ti ratios',
       'Effects of Al on Precipitation Behavior of Ti-Nb-Ta-Zr Refractory High Entropy Alloys',
       'Effect of Al addition on mechanical properties and microstructure of refractory AlxHfNbTaTiZr alloys',
       'High temperature strength of refractory complex concentrated alloys',
       'Microstructures and Crackling Noise of AlxNbTiMoV High Entropy Alloys',
       'Microstructure and Compressive Properties of NbTiVTaA


Present in alphabetical filtered paper df but not extracted
{'10.1007.s10853-019-03567-9', '10.1016.j.jallcom.2018.11.318'}


array(['A fine-grained NbMoTaWVCr refractory high-entropy alloy with ultra-high strength: Microstructural evolution and mechanical properties',
       'Correction to: Effect of Si additions on microstructure and mechanical properties of refractory NbTaWMo high-entropy alloys'],
      dtype=object)

In [77]:
first_20_papers

['A Novel Series of Refractory High-Entropy Alloys Ti2ZrHf0.5VNbx with High Specific Yield Strength and Good Ductility',
 'A fine-grained NbMoTaWVCr refractory high-entropy alloy with ultra-high strength: Microstructural evolution and mechanical properties',
 'A new refractory Ti-Nb-Hf-Al high entropy alloy strengthened by orthorhombic phase particles',
 'A novel HfNbTaTiV high-entropy alloy of superior mechanical properties designed on the principle of maximum lattice distortion',
 'A refractory Hf25Nb25Ti25Zr25 high-entropy alloy with excellent structural stability and tensile properties',
 'A simplified model connecting lattice distortion with friction stress of Nb-based equiatomic high-entropy alloys',
 'A single-phase V0.5Nb0.5ZrTi refractory high-entropy alloy with outstanding tensile properties',
 'Alloy Design and Properties Optimization of High-Entropy Alloys',
 'Alloy design for intrinsically ductile refractory high-entropy alloys',
 'An AlNbTiVZr0.5 high-entropy alloy combin

# Aligning extracted and ground-truth rows

In [78]:
# So at this point we have filtered_paper_df, consisting of the ground truth extracted values for the 20-paper sample,
# and extraction_df, consisting of the extracted values. colname_map maps from ground truth col names to extracted col names

In [79]:
target_cols = ['PROPERTY: HV',
               'PROPERTY: YS (MPa)',
               'PROPERTY: UTS (MPa)',
               'PROPERTY: Elongation (%)',
               'PROPERTY: Elongation plastic (%)',
               'PROPERTY: Exp. Young modulus (GPa)']

In [80]:
filtered_paper_df.columns

Index(['IDENTIFIER: Reference ID', 'FORMULA', 'PROPERTY: Microstructure',
       'PROPERTY: Processing method', 'PROPERTY: BCC/FCC/other',
       'PROPERTY: grain size ($\mu$m)', 'PROPERTY: Exp. Density (g/cm$^3$)',
       'PROPERTY: Computed Density (g/cm$^3$)', 'PROPERTY: HV',
       'PROPERTY: Type of test', 'PROPERTY: Test temperature ($^\circ$C)',
       'PROPERTY: YS (MPa)', 'PROPERTY: UTS (MPa)', 'PROPERTY: Elongation (%)',
       'PROPERTY: Elongation plastic (%)',
       'PROPERTY: Exp. Young modulus (GPa)',
       'PROPERTY: Computed Young modulus (GPa)', 'PROPERTY: O content (wppm)',
       'PROPERTY: N content (wppm)', 'PROPERTY: C content (wppm)',
       'REFERENCE: doi', 'REFERENCE: year', 'REFERENCE: title',
       'processed_doi'],
      dtype='object')

In [81]:
filtered_paper_df.iloc[0]

IDENTIFIER: Reference ID                                                                  7
FORMULA                                   Al0.025 Cr0.2 Mo0.2 Nb0.2 Ta0.025 Ti0.2 V0.05 ...
PROPERTY: Microstructure                                                          BCC+Laves
PROPERTY: Processing method                                                            CAST
PROPERTY: BCC/FCC/other                                                               other
PROPERTY: grain size ($\mu$m)                                                           NaN
PROPERTY: Exp. Density (g/cm$^3$)                                                      7.44
PROPERTY: Computed Density (g/cm$^3$)                                                   7.5
PROPERTY: HV                                                                            NaN
PROPERTY: Type of test                                                                    C
PROPERTY: Test temperature ($^\circ$C)                                          

In [82]:
extraction_df.iloc[0]

high entropy alloy formula                     MoNbFeCrV
yield strength                                      2663
doi                           10.1007.s40195-019-00935-x
grain size                                           NaN
experimental density                                 NaN
hardness                                             NaN
test temperature                                     NaN
ultimate tensile strength                            NaN
elongation                                           NaN
elongation plastic                                   NaN
Name: 0, dtype: object

## Unique records

In [83]:
paper_keys = ['FORMULA',
              'PROPERTY: Microstructure',
              'PROPERTY: Processing method',
              'PROPERTY: BCC/FCC/other',
              'PROPERTY: grain size ($\mu$m)',
              'PROPERTY: Type of test',
              'PROPERTY: Test temperature ($^\circ$C)']

extraction_keys = [colname_map[paper_key] for paper_key in paper_keys if paper_key in colname_map]
print(extraction_keys)

['high entropy alloy formula', 'grain size', 'test temperature']


I think we need to be trying to extract more keys from the data

We'll consider a unique record to be a unique combination of those 7 properties, and we will consider full matches and likely matches.

### normalize_formula() function

In [84]:
import pymatgen.core as mg


In [85]:
f1 = "Hf0.105 Nb0.053 Ti0.421 V0.211 Zr0.211"
f2 = "Ti2ZrHf0.5VNb0.25"


In [86]:
def rounding_fractions(comp_string: str) -> list[str]:
    elements = [f for i, f in enumerate(comp_string.split('$')) if i%2==0]
    fractions = [str(round(float(f.replace("_{", "").replace("}", "")), 3)) for i, f in enumerate(comp_string.split('$')) if
                 i % 2 != 0]
    final_results = [f"{i}{j}" for i, j in zip(elements, fractions)]
    return final_results

def normalize_comp_name(compName: str) -> str:
  try:
    comp = mg.Composition(compName)
    comp = mg.Composition(comp.alphabetical_formula)
    fractional_comp = comp.fractional_composition
    fract_comp_ustring = fractional_comp.to_unicode_string()
    normalized_frac_comp = rounding_fractions(fract_comp_ustring)
    normalized_comp_name = ''.join(f for f in normalized_frac_comp)

    return normalized_comp_name
  except:
    print(compName)
    return None


display(normalize_comp_name(f1))
display(normalize_comp_name(f2))

'Hf0.105Nb0.053Ti0.421V0.211Zr0.211'

'Hf0.105Nb0.053Ti0.421V0.211Zr0.211'

In [87]:
from traceback import print_exc
import re

float_pattern = '[0-9]*\.?[0-9]+'
def round_string_floats(s:str, precision:int=2):
  floats = re.search(float_pattern, s)


def normalize_formula(formula):
  try:
    return mg.Composition(formula).get_integer_formula_and_factor()[0]
  except Exception as ex:
    # print_exc()
    return None

f1 = "Hf0.105 Nb0.053 Ti0.421 V0.211 Zr0.211"
f2 = "Ti2ZrHf0.5VNb0.25"

display(normalize_formula(f1))
display(normalize_formula(f2))

round_string_floats(f1)

'Hf105Zr211Ti421Nb53V211'

'Hf2Zr4Ti8NbV4'

In [88]:
c1 = mg.Composition(f1)
display(c1)
c2 = mg.Composition(f2)
display(c2.fractional_composition)

display(c1.almost_equals(c2,rtol=0.1, atol=0.1))

Comp: Hf0.105 Zr0.211 Ti0.421 Nb0.053 V0.211

Comp: Hf0.10526316 Zr0.21052632 Ti0.42105263 Nb0.05263158 V0.21052632

False

In [89]:
c2 = mg.Composition(f2)
c2.fractional_composition

Comp: Hf0.10526316 Zr0.21052632 Ti0.42105263 Nb0.05263158 V0.21052632

In [90]:
# filtered_paper_df['normalized_formula'] = filtered_paper_df['FORMULA'].apply(normalize_formula)
filtered_paper_df['normalized_formula'] = filtered_paper_df['FORMULA'].apply(normalize_comp_name)

filtered_paper_df.sort_values('processed_doi')[['FORMULA','normalized_formula','processed_doi']]

/tmp/ipykernel_1783724/2496855811.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_paper_df['normalized_formula'] = filtered_paper_df['FORMULA'].apply(normalize_comp_name)


,FORMULA,normalized_formula,processed_doi
355,Fe0.167 Mo0.167 Ni0.167 Ti0.167 V0.167 Zr0.167,Fe0.167Mo0.167Ni0.167Ti0.167V0.167Zr0.167,10.1002.adem.200300567
823,Mo0.25 Re0.25 Ta0.25 W0.25,Mo0.25Re0.25Ta0.25W0.25,10.1002.adem.202100765
822,Mo0.25 Re0.25 Ta0.25 W0.25,Mo0.25Re0.25Ta0.25W0.25,10.1002.adem.202100765
631,Hf0.294 Ta0.118 Ti0.294 Zr0.294,Hf0.294Ta0.118Ti0.294Zr0.294,10.1002.adma.201701678
622,Hf0.25 Ta0.25 Ti0.25 Zr0.25,Hf0.25Ta0.25Ti0.25Zr0.25,10.1002.adma.201701678
...,...,...,...
428,Hf0.182 Nb0.182 Si0.091 Ti0.182 V0.182 Zr0.182,Hf0.182Nb0.182Si0.091Ti0.182V0.182Zr0.182,10.4028.www.scientific.net.MSF.849.76
196,Al0.2 Mo0.2 Ta0.2 Ti0.2 V0.2,Al0.2Mo0.2Ta0.2Ti0.2V0.2,10.4028.www.scientific.net.MSF.898.638
824,Mo0.25 Ta0.25 Ti0.25 V0.25,Mo0.25Ta0.25Ti0.25V0.25,10.4028.www.scientific.net.MSF.898.638
11,Al0.048 Mo0.238 Ta0.238 Ti0.238 V0.238,Al0.048Mo0.238Ta0.238Ti0.238V0.238,10.4028.www.scientific.net.MSF.898.638


In [91]:
# extraction_df['normalized_formula'] = extraction_df['high_entropy_alloy_formula'].apply(normalize_formula)
extraction_df['normalized_formula'] = extraction_df['high entropy alloy formula'].apply(normalize_comp_name)

extraction_df.sort_values('doi')[['high entropy alloy formula','normalized_formula','doi']]

nan
nan
{'Ti 20 Zr 20 Nb 20 Ta 20 Mo 20': {'grain size': 'No information', 'experimental density': 'No information', 'hardness': 4.9, 'test temperature': 'No information', 'yield strength': 1390, 'ultimate tensile strength': 'No information', 'elongation': 'No information', 'elongation plastic': 6}}
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
{'type': 'string', 'value': 'TiZrHfNbTa'}
{'type': 'string', 'value': 'TiZrHfNbTa'}
{'type': 'string', 'value': 'TiZrHfNbTa'}
{'type': 'string', 'value': 'TiZrHfNbTa'}
{'type': 'string', 'value': 'TiZrHfNbTa'}
{'type': 'string', 'value': 'TiZrHfNbTa'}
{'type': 'string', 'value': 'TiZrHfNbTa'}
{'type': 'string', 'value': 'TiZrHfNbTa'}
{'type': 'string', 'value': 'TiZrHfNbTa'}
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
Zr-Nb-Ti-Ta-Hf
Zr-Nb-Ti-Ta-Hf
Zr-Nb-Ti-Ta-Hf
Zr-Nb-Ti-Ta-Hf
Zr-Nb-Ti-Ta-Hf
Zr-Nb-Ti-Ta-Hf
Zr-Nb-Ti-Ta-Hf
Zr-Nb-Ti-Ta-

,high entropy alloy formula,normalized_formula,doi
653,CuCoNiCrAlFe,Al0.167Co0.167Cr0.167Cu0.167Fe0.167Ni0.167,10.1002.adem.200300567
654,CuCoNiCrAlFe,Al0.167Co0.167Cr0.167Cu0.167Fe0.167Ni0.167,10.1002.adem.200300567
655,CuCoNiCrAlFe,Al0.167Co0.167Cr0.167Cu0.167Fe0.167Ni0.167,10.1002.adem.200300567
656,CuCoNiCrAlFe,Al0.167Co0.167Cr0.167Cu0.167Fe0.167Ni0.167,10.1002.adem.200300567
657,CuCoNiCrAlFe,Al0.167Co0.167Cr0.167Cu0.167Fe0.167Ni0.167,10.1002.adem.200300567
...,...,...,...
318,HfNbSi0.5TiVZr,Hf0.182Nb0.182Si0.091Ti0.182V0.182Zr0.182,10.4028.www.scientific.net.MSF.849.76
807,VTaTiMoAl0,Mo0.25Ta0.25Ti0.25V0.25,10.4028.www.scientific.net.MSF.898.638
808,VTaTiMoAl0,Mo0.25Ta0.25Ti0.25V0.25,10.4028.www.scientific.net.MSF.898.638
809,VTaTiMoAl0,Mo0.25Ta0.25Ti0.25V0.25,10.4028.www.scientific.net.MSF.898.638


Basic logic: for each unique record in ground truth data, try to locate that record in the extracted data, as either:

Exact match between unique keys

Partial match between unique keys, with no other possible candidates

In [92]:
for target_col in target_cols:
  print('Target:',target_col)

Target: PROPERTY: HV
Target: PROPERTY: YS (MPa)
Target: PROPERTY: UTS (MPa)
Target: PROPERTY: Elongation (%)
Target: PROPERTY: Elongation plastic (%)
Target: PROPERTY: Exp. Young modulus (GPa)


## Row match functions

### assess_row_row_match()

In [93]:
import pandas as pd


from typing import Sequence, Dict, List


def assess_row_row_match(data_row: pd.Series,
                         extracted_row: pd.Series,
                         required_keys: Sequence[str],
                         other_keys: Sequence[str],
                         ):
    '''
    Assess the extent to which an extracted row matches a dataset row.
    Full if it matches all required and other keys
    Partial if it matches required keys and at least doesn't violate other keys (i.e. it has nulls where they have values)
    Nonmatch if it doesn't match required keys, or if violates any other keys (conflicting values)

    If we want to add flexible matching or unit conversion or something,
    (e.g. making '100mL' match '0.1L'), this is where to put it

    But nothing like this is currently implemented
    '''
    all_keys = required_keys + other_keys
    match_strength = 0
    for key in all_keys:
        if type(data_row[key]) == float and type(extracted_row[key]) == str:
          try:
            extracted_row[key] = float(extracted_row[key])
          except:
            pass
        if key in required_keys and extracted_row[key] != data_row[key]:
            return 0.0
        elif extracted_row[key] == data_row[key] or pd.isnull(extracted_row[key]) and pd.isnull(data_row[key]):
            match_strength +=1

    match_strength /= len(all_keys)
    return match_strength
    return result

In [94]:
r1 = filtered_paper_df.loc[375:375]
display(r1)

,IDENTIFIER: Reference ID,FORMULA,PROPERTY: Microstructure,PROPERTY: Processing method,PROPERTY: BCC/FCC/other,PROPERTY: grain size ($\mu$m),PROPERTY: Exp. Density (g/cm$^3$),PROPERTY: Computed Density (g/cm$^3$),PROPERTY: HV,PROPERTY: Type of test,PROPERTY: Test temperature ($^\circ$C),PROPERTY: YS (MPa),PROPERTY: UTS (MPa),PROPERTY: Elongation (%),PROPERTY: Elongation plastic (%),PROPERTY: Exp. Young modulus (GPa),PROPERTY: Computed Young modulus (GPa),PROPERTY: O content (wppm),PROPERTY: N content (wppm),PROPERTY: C content (wppm),REFERENCE: doi,REFERENCE: year,REFERENCE: title,processed_doi,normalized_formula
375,142,Hf0.105 Nb0.053 Ti0.421 V0.211 Zr0.211,BCC,CAST,BCC,NaN,NaN,6.4,332.4,C,25.0,1115.0,NaN,50.0,NaN,NaN,104.0,NaN,NaN,NaN,10.1007/s40195-019-00921-3,2019.0,A Novel Series of Refractory High-Entropy Allo...,10.1007.s40195-019-00921-3,Hf0.105Nb0.053Ti0.421V0.211Zr0.211


In [95]:
r2 = extraction_df.loc[1:1].rename(columns=inverted_colname_map)
display(r2)

,FORMULA,PROPERTY: YS (MPa),REFERENCE: doi,PROPERTY: grain size ($\mu$m),PROPERTY: Exp. Density (g/cm$^3$),PROPERTY: HV,PROPERTY: Test temperature ($^\circ$C),PROPERTY: UTS (MPa),PROPERTY: Elongation (%),PROPERTY: Elongation plastic (%),normalized_formula
1,MoNbFeCrV,2663,10.1007.s40195-019-00935-x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cr0.2Fe0.2Mo0.2Nb0.2V0.2


In [96]:
assess_row_row_match(data_row=r1.iloc[0],
                         extracted_row=r2.iloc[0],
                         required_keys=['normalized_formula', 'PROPERTY: YS (MPa)'],
                         other_keys= [
                            #  'PROPERTY: Microstructure',
              # 'PROPERTY: Processing method',
              # 'PROPERTY: BCC/FCC/other',
              'PROPERTY: grain size ($\mu$m)',
              # 'PROPERTY: Type of test',
              'PROPERTY: Test temperature ($^\circ$C)'])

0.0

### assess_df_df_match()

In [97]:
from typing import List, Tuple

def assess_df_df_matches(
        data_df: pd.DataFrame,  # test df
        extracted_df: pd.DataFrame,  # extracted df
        required_keys: List[str],
        other_keys: List[str]
):
    '''
    Assess pairwise row matches between two dfs. Assume columns match.
    '''
    extracted_data_matches = {}

    for extracted_idx, extracted_row in extracted_df.iterrows():
        extracted_data_matches[extracted_idx] = {}
        for data_idx, data_row in data_df.iterrows():

            row_match = assess_row_row_match(data_row=data_row,
                                             extracted_row=extracted_row,
                                             required_keys=required_keys, other_keys=other_keys)
            extracted_data_matches[extracted_idx][data_idx] = row_match

    return extracted_data_matches


### perform_matching()

In [98]:
def perform_matching(extracted_data_matches:Dict[int, Dict[int,float]], exclusive_threshold=0.2, verbose:bool=False):
    '''
    Given a dictionary of row-to-row match strength values, determine a likely one-to-one pairing of rows
    by greedily picking the strongest match until there are no more unmatched rows or until no matches are stronger
    than the threshold value

    :param extracted_data_matches:
    :param data_extracted_matches:
    :return:
    '''
    unmatched_extracted_idxs = set()
    unmatched_data_idxs = set()
    match_strengths = []

    for extracted_idx, match_dict in extracted_data_matches.items():
        for data_idx, match_strength in match_dict.items():
            match_strengths.append((extracted_idx, data_idx, match_strength))
            unmatched_extracted_idxs.add(extracted_idx)
            unmatched_data_idxs.add(data_idx)

    match_strengths = sorted(match_strengths, key=lambda m:m[2], reverse=True)
    if verbose:
      print(match_strengths)

    matches= []

    for extracted_idx, data_idx, match_strength in match_strengths:
        if match_strength > exclusive_threshold and extracted_idx in unmatched_extracted_idxs and data_idx in unmatched_data_idxs:
            matches.append((extracted_idx, data_idx, match_strength))
            unmatched_extracted_idxs.remove(extracted_idx)
            unmatched_data_idxs.remove(data_idx)

    return matches, list(unmatched_extracted_idxs), list(unmatched_data_idxs)


### align_dfs()

In [99]:
from typing import Union, Callable
def align_dfs(data_df: pd.DataFrame,  # test df
                      extracted_df: pd.DataFrame,  # extracted df
                      required_keys: List[str],  # keys rows absolutely must match on to be a potential match (e.g. formula, target value (yield strength, etc))
                      other_keys: List[str],  # keys rows could/should match on if possible, but a non-match isn't disqualifying (e.g. grain size, test temp)
                      extracted_to_data_colmap: Dict[str, str],  # map between data and extracted df columns
                      group_by: Union[str, Tuple[str,str]],  # group by this before doing the matching. Should be paper title or DOI. If tuple, first is for data_df, second for extracted_df
                      discard_nonextracted_keys: bool = False,  # If true, ignore other_keys that don't exist in the extracted_df
                      verbose: bool = False,
                      debug_group_count: int=None,
                      debug_group_skip:int=None,
                      debug_group_filter:Callable=None
                      ):
    '''
    :param data_df: human-annotated df
   :param extracted_df: model-extracted df. Should have same structure as human-annotated one, but columns don't have to match
   :param required_keys: keys that absolutely have to match for a pair of records to be considered a potential match (e.g. formula and YS, if YS is the target value)
   :param other_keys: keys that should match, but don't absolutely have to (e.g. grain size, test temp)
   :param extracted_to_data_colmap: dictionary mapping column names from extracted DF to original DF
   :param group_by: column to group by before doing the matching. Should be title or DOI
   :param discard_nonextracted_keys: whether to ignore any keys that weren't extracted, as these could never be a match (e.g. test type if we didn't try to extract it)
   :param verbose: whether to print additional output
   :return:

    Take a dataframe of human-extracted values (e.g. Carolina's dataset), and align it with
    a dataframe of model-extracted values.

    Goal here is alignment--figure out which extracted records match with which original records,
    and which original and extracted records are unmatched.

    Do this with a greedy matching algorithm.

    Return a combined dataframe where matched records are merged side by side with their columns
    suffixed with '_original' and '_extracted' respectively. Unmatched records are included too, with
    NaN values for the columns that couldn't be matched.

   index  extracted_idx  original_idx  match_strength match_group formula_original  test_temp_original  grain_size_original test_type_original  ys_original title_original formula_extracted  test_temp_extracted grain_size_extracted  ys_extracted title_extracted
0      0            0.0           0.0            0.75     paper_1              NbV               100.0                 25.5             Normal         14.4        paper_1               NbV                100.0                 None          14.4         paper_1
1      1            NaN           1.0             NaN     paper_1              NbV               200.0                 34.0             Normal         24.8        paper_1               NaN                  NaN                  NaN           NaN             NaN
2      1            1.0           NaN             NaN     paper_1              NaN                 NaN                  NaN                NaN          NaN            NaN               NbV                300.0                 None          24.5         paper_1

    So when you want to isolate rows that were successfully matched, just look for pd.notnull(original_idx) and pd.notnull(extracted_idx), and
    vice versa for finding original or extracted rows that couldn't be matched.

    '''
    all_keys = required_keys + other_keys
    data_keys = [col for col in all_keys if col in data_df]

    if verbose:
        print('True data DF:')
        display(data_df[data_keys])

    extracted_df = extracted_df.rename(columns=extracted_to_data_colmap)

    if discard_nonextracted_keys:
        other_keys = [key for key in other_keys if key in extracted_df]

    extracted_keys  = [key for key in all_keys if key in extracted_df]

    if verbose:
        print('Extracted data DF (with renamed columns):')
        display(extracted_df[extracted_keys])
        print('Required keys:', required_keys)
        print('Other keys to match on:', other_keys)
        print('\nMatching rows:')

    if type(group_by) == str:
      data_groups = data_df.groupby(group_by)
      extracted_groups = extracted_df.groupby(group_by)
    elif type(group_by) == tuple:
      data_groups = data_df.groupby(group_by[0])
      extracted_groups = extracted_df.groupby(group_by[1])
      group_by=group_by[0]

    # By definition we can assume that data_df has a nonempty group for every paper
    # The same is not necessarily true for extracted_df, if no values were found for a paper

    match_dfs = []
    group_count = 0
    group_skip = 0 if debug_group_skip is None else debug_group_skip

    for group, data_indices in data_groups.indices.items():
        extracted_indices = extracted_groups.indices[group] if group in extracted_groups.indices else []

        if verbose:
          print('---')
          print(group_by, group)
          print('original indices',data_indices)
          print('extracted indices', extracted_indices)

        group_data_df = data_df.iloc[data_indices]
        group_extracted_df = extracted_df.iloc[extracted_indices]

        if debug_group_filter and ( not debug_group_filter(group_data_df) or not debug_group_filter(group_extracted_df)):
          if verbose: print('Group debug filter fail. Skipping.')
          continue

        group_count +=1
        if group_count <= group_skip:
          if verbose:print('Skipping for debug')
          continue

        if verbose:
          print('Group data df:')
          display(group_data_df[data_keys])
          print('Group extracted df:')
          display(group_extracted_df[extracted_keys])

        e_d_matches = assess_df_df_matches(data_df=group_data_df,
                                                        extracted_df=group_extracted_df,
                                                        required_keys=required_keys,
                                                        other_keys=other_keys)

        # print('extracted matches')
        # print(e_d_matches)

        matches, unmatched_extracted_idxs, unmatched_data_idxs = perform_matching(extracted_data_matches=e_d_matches, verbose=verbose)


        match_df = pd.DataFrame(matches, columns=['extracted_idx','original_idx', 'match_strength'])
        match_df['match_group'] = group

        if verbose:
          print('matches:')
          print(matches)
          print('match df:')
          display(match_df)


        match_df = match_df.merge(group_data_df.rename(columns=lambda c:c+'_original'), left_on='original_idx', right_index=True)

        if verbose:
          print('merged match df:')
          display(match_df)

        match_df = match_df.merge(group_extracted_df.rename(columns=lambda c: c + '_extracted'), left_on='extracted_idx', right_index=True)

        if verbose:
          print('second merged match df:')
          display(match_df)

        unmatched_extracted_df = group_extracted_df.loc[unmatched_extracted_idxs].rename(columns=lambda c:c+'_extracted')
        unmatched_extracted_df['extracted_idx'] = unmatched_extracted_idxs
        unmatched_extracted_df['match_group'] = group

        unmatched_data_df = group_data_df.loc[unmatched_data_idxs].rename(columns=lambda c:c+'_original')
        unmatched_data_df['original_idx'] = unmatched_data_idxs
        unmatched_data_df['match_group'] = group

        match_df = pd.concat([match_df, unmatched_data_df, unmatched_extracted_df], axis=0)
        match_dfs.append(match_df)
        pass

        if verbose:
          print('===')

        if debug_group_count and group_count >= debug_group_count+group_skip:
          break

    all_match_df = pd.concat(match_dfs, axis=0).reset_index()
    return all_match_df


### Testing

#### Toy dataset

In [100]:
required_keys = ['formula', 'ys']
other_keys = [ 'test_temp', 'grain_size', 'test_type']

t_tdf = pd.DataFrame([{'formula':'NbV', 'test_temp':100, 'grain_size':25.5, 'test_type':'Normal', 'ys':14.4, 'title':'paper_1'},
                     {'formula':'NbV', 'test_temp':200, 'grain_size':34, 'test_type':'Normal', 'ys':24.8, 'title':'paper_1'},
                     ])


tkey_mapping = {'test_temp':'test temperature', 'grain_size':'grain size', 'ys':'yield strength'}
inverted_tkey_mapping = {value:key for key, value in tkey_mapping.items()}

t_edf1 = pd.DataFrame([{'formula':'NbV', 'test temperature':100, 'grain size':None, 'yield strength':14.4, 'title':'paper_1'},
                       {'formula':'NbV', 'test temperature':300, 'grain size':None, 'yield strength':24.5, 'title':'paper_1'},
                       ])




df_result = align_dfs(data_df=t_tdf,
                  extracted_df=t_edf1,
                  required_keys=required_keys,
                  other_keys=other_keys,
                  extracted_to_data_colmap=inverted_tkey_mapping,
                  group_by='title',
                  discard_nonextracted_keys=True,
                  verbose=True
                  )

pd.options.display.width=300
print(df_result)

True data DF:


,formula,ys,test_temp,grain_size,test_type
0,NbV,14.4,100,25.5,Normal
1,NbV,24.8,200,34.0,Normal


Extracted data DF (with renamed columns):


,formula,ys,test_temp,grain_size
0,NbV,14.4,100,None
1,NbV,24.5,300,None


Required keys: ['formula', 'ys']
Other keys to match on: ['test_temp', 'grain_size']

Matching rows:
---
title paper_1
original indices [0 1]
extracted indices [0 1]
Group data df:


...

merged match df:


,extracted_idx,original_idx,match_strength,match_group,formula_original,test_temp_original,grain_size_original,test_type_original,ys_original,title_original
0,0,0,0.75,paper_1,NbV,100,25.5,Normal,14.4,paper_1


second merged match df:


,extracted_idx,original_idx,match_strength,match_group,formula_original,test_temp_original,grain_size_original,test_type_original,ys_original,title_original,formula_extracted,test_temp_extracted,grain_size_extracted,ys_extracted,title_extracted
0,0,0,0.75,paper_1,NbV,100,25.5,Normal,14.4,paper_1,NbV,100,None,14.4,paper_1


===
   index  extracted_idx  original_idx  match_strength match_group formula_original  test_temp_original  grain_size_original test_type_original  ys_original title_original formula_extracted  test_temp_extracted grain_size_extracted  ys_extracted title_extracted
0      0            0.0           0.0            0.75     paper_1              NbV               100.0                 25.5             Normal         14.4        paper_1               NbV                100.0                 None          14.4         paper_1
1      1            NaN           1.0             NaN     paper_1              NbV               200.0                 34.0             Normal         24.8        paper_1               NaN                  NaN                  NaN           NaN             NaN
2      1            1.0           NaN             NaN     paper_1              NaN                 NaN                  NaN                NaN          NaN            NaN               NbV                300.0    

#### Real data

In [101]:
# These have to be a match
required_keys = ['normalized_formula', # I added this to both dfs up above in the unique records section
                 'PROPERTY: YS (MPa)']

# These should be a match but it isn't the end of the world if they aren't
other_keys =[ 'PROPERTY: Microstructure',
              'PROPERTY: Processing method',
              'PROPERTY: BCC/FCC/other',
              'PROPERTY: grain size ($\mu$m)',
              'PROPERTY: Type of test',
              'PROPERTY: Test temperature ($^\circ$C)']


df_result = align_dfs(data_df=filtered_paper_df,
                  extracted_df=extraction_df,
                  required_keys=required_keys,
                  other_keys=other_keys,
                  extracted_to_data_colmap=inverted_colname_map,
                  group_by=('processed_doi', 'REFERENCE: doi'),
                  discard_nonextracted_keys=True,
                  # verbose=True,
                  # debug_group_count=1,
                  # debug_group_skip=1,
                  # debug_group_filter = lambda df: np.any(df['PROPERTY: YS (MPa)'].notnull())
                  )



display(df_result)

,index,extracted_idx,original_idx,match_strength,match_group,IDENTIFIER: Reference ID_original,FORMULA_original,PROPERTY: Microstructure_original,PROPERTY: Processing method_original,PROPERTY: BCC/FCC/other_original,PROPERTY: grain size ($\mu$m)_original,PROPERTY: Exp. Density (g/cm$^3$)_original,PROPERTY: Computed Density (g/cm$^3$)_original,PROPERTY: HV_original,PROPERTY: Type of test_original,PROPERTY: Test temperature ($^\circ$C)_original,PROPERTY: YS (MPa)_original,PROPERTY: UTS (MPa)_original,PROPERTY: Elongation (%)_original,PROPERTY: Elongation plastic (%)_original,PROPERTY: Exp. Young modulus (GPa)_original,PROPERTY: Computed Young modulus (GPa)_original,PROPERTY: O content (wppm)_original,PROPERTY: N content (wppm)_original,PROPERTY: C content (wppm)_original,REFERENCE: doi_original,REFERENCE: year_original,REFERENCE: title_original,processed_doi_original,normalized_formula_original,FORMULA_extracted,PROPERTY: YS (MPa)_extracted,REFERENCE: doi_extracted,PROPERTY: grain size ($\mu$m)_extracted,PROPERTY: Exp. Density (g/cm$^3$)_extracted,PROPERTY: HV_extracted,PROPERTY: Test temperature ($^\circ$C)_extracted,PROPERTY: UTS (MPa)_extracted,PROPERTY: Elongation (%)_extracted,PROPERTY: Elongation plastic (%)_extracted,normalized_formula_extracted
0,355,NaN,355.0,NaN,10.1002.adem.200300567,15.0,Fe0.167 Mo0.167 Ni0.167 Ti0.167 V0.167 Zr0.167,BCC+Sec.,CAST,other,NaN,NaN,7.4,740.0,NaN,25.0,NaN,NaN,NaN,NaN,NaN,175.0,NaN,NaN,NaN,10.1002/adem.200300567,2004.0,Nanostructured High-Entropy Alloys with Multip...,10.1002.adem.200300567,Fe0.167Mo0.167Ni0.167Ti0.167V0.167Zr0.167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,653,653.0,NaN,NaN,10.1002.adem.200300567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CuCoNiCrAlFe,NaN,10.1002.adem.200300567,NaN,NaN,133,NaN,NaN,NaN,NaN,Al0.167Co0.167Cr0.167Cu0.167Fe0.167Ni0.167
2,654,654.0,NaN,NaN,10.1002.adem.200300567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CuCoNiCrAlFe,NaN,10.1002.adem.200300567,NaN,NaN,133,NaN,NaN,NaN,NaN,Al0.167Co0.167Cr0.167Cu0.167Fe0.167Ni0.167
3,655,655.0,NaN,NaN,10.1002.adem.200300567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CuCoNiCrAlFe,NaN,10.1002.adem.200300567,NaN,NaN,133,NaN,NaN,NaN,NaN,Al0.167Co0.167Cr0.167Cu0.167Fe0.167Ni0.167
4,656,656.0,NaN,NaN,10.1002.adem.200300567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CuCoNiCrAlFe,NaN,10.1002.adem.200300567,NaN,NaN,133,NaN,NaN,NaN,NaN,Al0.167Co0.167Cr0.167Cu0.167Fe0.167Ni0.167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1749,11,NaN,11.0,NaN,10.4028.www.scientific.net.MSF.898.638,86.0,Al0.048 Mo0.238 Ta0.238 Ti0.238 V0.238,BCC,CAST,BCC,NaN,NaN,9.1,NaN,C,25.0,1021.0,1249.0,NaN,NaN,NaN,184.0,NaN,NaN,NaN,10.4028/www.scientific.net/MSF.898.638,2017.0,Microstructure and Mechanical Properties of VT...,10.4028.www.scientific.net.MSF.898.638,Al0.048Mo0.238Ta0.238Ti0.238V0.238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1750,196,NaN,196.0,NaN,10.4028.www.scientific.net.MSF.898.638,86.0,Al0.2 Mo0.2 Ta0.2 Ti0.2 V0.2,BCC,CAST,BCC,NaN,NaN,8.0,NaN,C,25.0,NaN,735.0,NaN,NaN,NaN,166.0,NaN,NaN,NaN,10.4028/www.scientific.net/MSF.898.638,2017.0,Microstructure and Mechanical Properties of VT...,10.4028.www.scientific.net.MSF.898.638,Al0.2Mo0.2Ta0.2Ti0.2V0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1751,808,808.0,NaN,NaN,10.4028.www.scientific.net.MSF.898.638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VTaTiMoAl0,1221.0,10.4028.www.scientific.net.MSF.898.638,NaN,NaN,NaN,NaN,1454.0,9.93,NaN,Mo0.25Ta0.25Ti0.25V0.25
1752,809,809.0,NaN,NaN,10.4028.www.scientific.net.MSF.898.638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

##### Looking at matched rows

In [102]:
match_df = df_result[df_result['original_idx'].notnull() & df_result['extracted_idx'].notnull()]
print(match_df.shape)
display(match_df)

(52, 41)


,index,extracted_idx,original_idx,match_strength,match_group,IDENTIFIER: Reference ID_original,FORMULA_original,PROPERTY: Microstructure_original,PROPERTY: Processing method_original,PROPERTY: BCC/FCC/other_original,PROPERTY: grain size ($\mu$m)_original,PROPERTY: Exp. Density (g/cm$^3$)_original,PROPERTY: Computed Density (g/cm$^3$)_original,PROPERTY: HV_original,PROPERTY: Type of test_original,PROPERTY: Test temperature ($^\circ$C)_original,PROPERTY: YS (MPa)_original,PROPERTY: UTS (MPa)_original,PROPERTY: Elongation (%)_original,PROPERTY: Elongation plastic (%)_original,PROPERTY: Exp. Young modulus (GPa)_original,PROPERTY: Computed Young modulus (GPa)_original,PROPERTY: O content (wppm)_original,PROPERTY: N content (wppm)_original,PROPERTY: C content (wppm)_original,REFERENCE: doi_original,REFERENCE: year_original,REFERENCE: title_original,processed_doi_original,normalized_formula_original,FORMULA_extracted,PROPERTY: YS (MPa)_extracted,REFERENCE: doi_extracted,PROPERTY: grain size ($\mu$m)_extracted,PROPERTY: Exp. Density (g/cm$^3$)_extracted,PROPERTY: HV_extracted,PROPERTY: Test temperature ($^\circ$C)_extracted,PROPERTY: UTS (MPa)_extracted,PROPERTY: Elongation (%)_extracted,PROPERTY: Elongation plastic (%)_extracted,normalized_formula_extracted
27,0,373.0,639.0,0.75,10.1007.s00339-019-2484-1,15.0,Hf0.333 Ti0.333 Zr0.333,Other,CAST,other,NaN,NaN,8.1,211.00,C,25.0,773.0,NaN,17.0,NaN,NaN,87.0,NaN,NaN,NaN,10.1007/s00339-019-2484-1,2019.0,Effect of Sc and Y addition on the microstruct...,10.1007.s00339-019-2484-1,Hf0.333Ti0.333Zr0.333,TiZrHf,773.0,10.1007.s00339-019-2484-1,NaN,NaN,211.0,NaN,NaN,17.0,NaN,Hf0.333Ti0.333Zr0.333
39,0,20.0,465.0,0.50,10.1007.s10853-012-6260-2,53.0,Hf0.2 Nb0.2 Ta0.2 Ti0.2 Zr0.2,BCC,OTHER,BCC,NaN,NaN,9.9,NaN,C,25.0,929.0,NaN,50.0,NaN,NaN,111.0,NaN,NaN,NaN,10.1007/s10853-012-6260-2,2012.0,Microstructure and elevated temperature proper...,10.1007.s10853-012-6260-2,Hf0.2Nb0.2Ta0.2Ti0.2Zr0.2,Ta 20 Nb 20 Hf 20 Zr 20 Ti 20,929.0,10.1007.s10853-012-6260-2,100.0,9.94,NaN,296.0,NaN,NaN,50.0,Hf0.2Nb0.2Ta0.2Ti0.2Zr0.2
107,0,115.0,190.0,1.00,10.1007.s11837-014-1066-0,39.0,Al0.2 Mo0.1 Nb0.2 Ta0.1 Ti0.2 Zr0.2,BCC+B2,OTHER,other,75.000,NaN,7.2,591.00,C,25.0,2000.0,2368.0,10.0,NaN,NaN,123.0,NaN,NaN,NaN,10.1007/s11837-014-1066-0,2014.0,Microstructure and Properties of Aluminum-Cont...,10.1007.s11837-014-1066-0,Al0.2Mo0.1Nb0.2Ta0.1Ti0.2Zr0.2,AlMo 0.5 NbTa 0.5 TiZr,2000.0,10.1007.s11837-014-1066-0,75.0,7.4,5.8,25.0,2368.0,10.0,NaN,Al0.2Mo0.1Nb0.2Ta0.1Ti0.2Zr0.2
148,0,431.0,287.0,0.75,10.1007.s11837-020-04557-y,23.0,Cr0.2 Hf0.2 Nb0.2 Ta0.2 Ti0.2,BCC+BCC+Laves,CAST,other,NaN,NaN,10.0,NaN,C,25.0,1258.0,NaN,24.3,NaN,NaN,153.0,NaN,NaN,NaN,10.1007/s11837-020-04557-y,2021.0,Microstructure and Mechanical Properties of a ...,10.1007.s11837-020-04557-y,Cr0.2Hf0.2Nb0.2Ta0.2Ti0.2,CrHfNbTaTi,1258.0,10.1007.s11837-020-04557-y,NaN,NaN,NaN,296.0,2061,24.3,NaN,Cr0.2Hf0.2Nb0.2Ta0.2Ti0.2
175,0,603.0,378.0,0.75,10.1007.s40195-019-00921-3,142.0,Hf0.111 Ti0.444 V0.222 Zr0.222,BCC,CAST,BCC,NaN,NaN,6.3,338.30,C,25.0,1160.0,NaN,50.0,NaN,NaN,104.0,NaN,NaN,NaN,10.1007/s40195-019-00921-3,2019.0,A Novel Series of Refractory High-Entropy Allo...,10.1007.s40195-019-00921-3,Hf0.111Ti0.444V0.222Zr0.222,Ti2ZrHf0.5VNb0,1160,10.1007.s40195-019-00921-3,NaN,6.47,338,298,NaN,>50,NaN,Hf0.111Ti0.444V0.222Zr0.222
194,0,0.0,286.0,0.75,10.1007.s40195-019-00935-x,136.0,Cr0.2 Fe0.2 Mo0.2 Nb0.2 V0.2,BCC+Laves,CAST,other,NaN,NaN,8.0,826.00,C,25.0,2663.0,NaN,2.7,NaN,NaN,210.0,NaN,NaN,NaN,10.1007/s40195-019-00935-x,2019.0,"Microstructure, Mechanical Properties, and Cor...",10.1007.s40195-019-00935-x,Cr0.2Fe0.2Mo0.2Nb0.2V0.2,MoNbFeCrV,2663,10.1007.s40195-019-00935-x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cr0.2Fe0.2Mo0.2Nb0.2V0.2
217,0,235.0,603.0,0.75,10.1016.j.actamat.2016.01.018,87.0,Hf0.25 Nb0.25 Ta0.25 Zr0.25,BCC,CAST,BCC,NaN,NaN,11.3,365.00,C,25.0,1315.0,1800.0,21.6,NaN,NaN,109.0,NaN,NaN,NaN,10.1016/j.actamat.2016.01.018,2016.0,Structural-disorder and its effect on mechanic

##### Looking at original rows

In [103]:
original_df = df_result[df_result['original_idx'].notnull()]

In [104]:
len(original_df[original_df['PROPERTY: YS (MPa)_original'].notnull()])

817

In [105]:
# And then we can evaluated p/r/f1 for individual columns
# Finding that scikit-learn is broken for some reason
# from sklearn.metrics import accuracy_score
# classification_report(original_df['PROPERTY: YS (MPa)_original'], original_df['PROPERTY: YS (MPa)_extracted'])

##### Comparing match rows for a particular paper

In [106]:
nonmatch_df = df_result[df_result['original_idx'].isnull() | df_result['extracted_idx'].isnull()]
print(nonmatch_df.shape)


(1702, 41)


In [107]:
match_df = df_result[df_result['original_idx'].notnull() & df_result['extracted_idx'].notnull()]
print(match_df.shape)

(52, 41)


In [109]:
match_df.to_csv('matched_simple_zeroshot.csv', encoding='utf8')

In [110]:
original_df = df_result[df_result['original_idx'].notnull()]
print(original_df.shape)

(946, 41)


In [111]:
groups = df_result.groupby('match_group').indices
groups

{'10.1002.adem.200300567': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 '10.1002.adem.202100765': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]),
 '10.1002.adma.201701678': array([21, 22, 23, 24, 25, 26]),
 '10.1007.s00339-019-2484-1': array([27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]),
 '10.1007.s10853-012-6260-2': array([39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52]),
 '10.1007.s11661-017-4386-1': array([53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68]),
 '10.1007.s11661-018-4646-8': array([69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82]),
 '10.1007.s11665-017-2799-z': array([83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96]),
 '10.1007.s11837-012-0366-5': array([ 97,  98,  99, 100, 101, 102, 103, 104, 105, 106]),
 '10.1007.s11837-014-1066-0': array([107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119,
        120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132,
        133]),
 '10.1007.s11837-019-03432-9': array([13

In [112]:
#Taking a look at the non-matched original versus extracted rows for an individual paper
doi = list(groups.keys())[2]

print('Matched rows:')
display(match_df[match_df['match_group']==doi][['normalized_formula_original',
                                                     'normalized_formula_extracted',
                                                     'FORMULA_original',
                                                     'FORMULA_extracted',
                                                     'PROPERTY: YS (MPa)_original',
                                                     'PROPERTY: YS (MPa)_extracted',
                                                     'match_group']])

print('Non-matched rows:')
display(nonmatch_df[nonmatch_df['match_group']==doi][['original_idx', 'extracted_idx',
                                                      'normalized_formula_original',
                                                     'normalized_formula_extracted',
                                                     'FORMULA_original',
                                                     'FORMULA_extracted',
                                                     'PROPERTY: YS (MPa)_original',
                                                     'PROPERTY: YS (MPa)_extracted',
                                                     'match_group']])

Matched rows:


,normalized_formula_original,normalized_formula_extracted,FORMULA_original,FORMULA_extracted,PROPERTY: YS (MPa)_original,PROPERTY: YS (MPa)_extracted,match_group


Non-matched rows:


,original_idx,extracted_idx,normalized_formula_original,normalized_formula_extracted,FORMULA_original,FORMULA_extracted,PROPERTY: YS (MPa)_original,PROPERTY: YS (MPa)_extracted,match_group
21,627.0,NaN,Hf0.278Ta0.167Ti0.278Zr0.278,NaN,Hf0.278 Ta0.167 Ti0.278 Zr0.278,NaN,750.0,NaN,10.1002.adma.201701678
22,628.0,NaN,Hf0.286Ta0.143Ti0.286Zr0.286,NaN,Hf0.286 Ta0.143 Ti0.286 Zr0.286,NaN,687.0,NaN,10.1002.adma.201701678
23,622.0,NaN,Hf0.25Ta0.25Ti0.25Zr0.25,NaN,Hf0.25 Ta0.25 Ti0.25 Zr0.25,NaN,1356.0,NaN,10.1002.adma.201701678
24,631.0,NaN,Hf0.294Ta0.118Ti0.294Zr0.294,NaN,Hf0.294 Ta0.118 Ti0.294 Zr0.294,NaN,354.0,NaN,10.1002.adma.201701678
25,NaN,545.0,NaN,Hf0.25Ta0.25Ti0.25Zr0.25,NaN,Ta1HfZrTi,NaN,1500,10.1002.adma.201701678
26,NaN,546.0,NaN,Hf0.25Ta0.25Ti0.25Zr0.25,NaN,Ta1HfZrTi,NaN,1500,10.1002.adma.201701678


In [113]:
extraction_df[extraction_df['doi'] ==doi]

,high entropy alloy formula,yield strength,doi,grain size,experimental density,hardness,test temperature,ultimate tensile strength,elongation,elongation plastic,normalized_formula
545,Ta1HfZrTi,1500,10.1002.adma.201701678,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hf0.25Ta0.25Ti0.25Zr0.25
546,Ta1HfZrTi,1500,10.1002.adma.201701678,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hf0.25Ta0.25Ti0.25Zr0.25


In [114]:
df_result[df_result['match_group']==doi][['original_idx', 'extracted_idx',
                                          'normalized_formula_original',
                                                     'normalized_formula_extracted',
                                                     'FORMULA_original',
                                                     'FORMULA_extracted',
                                                     'PROPERTY: YS (MPa)_original',
                                                     'PROPERTY: YS (MPa)_extracted',
                                                     'match_group']]

,original_idx,extracted_idx,normalized_formula_original,normalized_formula_extracted,FORMULA_original,FORMULA_extracted,PROPERTY: YS (MPa)_original,PROPERTY: YS (MPa)_extracted,match_group
21,627.0,NaN,Hf0.278Ta0.167Ti0.278Zr0.278,NaN,Hf0.278 Ta0.167 Ti0.278 Zr0.278,NaN,750.0,NaN,10.1002.adma.201701678
22,628.0,NaN,Hf0.286Ta0.143Ti0.286Zr0.286,NaN,Hf0.286 Ta0.143 Ti0.286 Zr0.286,NaN,687.0,NaN,10.1002.adma.201701678
23,622.0,NaN,Hf0.25Ta0.25Ti0.25Zr0.25,NaN,Hf0.25 Ta0.25 Ti0.25 Zr0.25,NaN,1356.0,NaN,10.1002.adma.201701678
24,631.0,NaN,Hf0.294Ta0.118Ti0.294Zr0.294,NaN,Hf0.294 Ta0.118 Ti0.294 Zr0.294,NaN,354.0,NaN,10.1002.adma.201701678
25,NaN,545.0,NaN,Hf0.25Ta0.25Ti0.25Zr0.25,NaN,Ta1HfZrTi,NaN,1500,10.1002.adma.201701678
26,NaN,546.0,NaN,Hf0.25Ta0.25Ti0.25Zr0.25,NaN,Ta1HfZrTi,NaN,1500,10.1002.adma.201701678


In [115]:
filtered_paper_df[filtered_paper_df['processed_doi'] ==doi]

,IDENTIFIER: Reference ID,FORMULA,PROPERTY: Microstructure,PROPERTY: Processing method,PROPERTY: BCC/FCC/other,PROPERTY: grain size ($\mu$m),PROPERTY: Exp. Density (g/cm$^3$),PROPERTY: Computed Density (g/cm$^3$),PROPERTY: HV,PROPERTY: Type of test,PROPERTY: Test temperature ($^\circ$C),PROPERTY: YS (MPa),PROPERTY: UTS (MPa),PROPERTY: Elongation (%),PROPERTY: Elongation plastic (%),PROPERTY: Exp. Young modulus (GPa),PROPERTY: Computed Young modulus (GPa),PROPERTY: O content (wppm),PROPERTY: N content (wppm),PROPERTY: C content (wppm),REFERENCE: doi,REFERENCE: year,REFERENCE: title,processed_doi,normalized_formula
622,88,Hf0.25 Ta0.25 Ti0.25 Zr0.25,BCC,CAST,BCC,NaN,NaN,10.2,NaN,T,25.0,1356.0,1452.0,4.0,NaN,NaN,112.0,NaN,NaN,NaN,10.1002/adma.201701678,2017.0,Phase-Transformation Ductilization of Brittle ...,10.1002.adma.201701678,Hf0.25Ta0.25Ti0.25Zr0.25
627,88,Hf0.278 Ta0.167 Ti0.278 Zr0.278,BCC+HCP,CAST,other,NaN,NaN,9.5,NaN,T,25.0,750.0,1110.0,NaN,NaN,NaN,104.0,NaN,NaN,NaN,10.1002/adma.201701678,2017.0,Phase-Transformation Ductilization of Brittle ...,10.1002.adma.201701678,Hf0.278Ta0.167Ti0.278Zr0.278
628,88,Hf0.286 Ta0.143 Ti0.286 Zr0.286,BCC+HCP,CAST,other,NaN,NaN,9.3,NaN,T,25.0,687.0,1119.0,29.9,NaN,NaN,101.0,NaN,NaN,NaN,10.1002/adma.201701678,2017.0,Phase-Transformation Ductilization of Brittle ...,10.1002.adma.201701678,Hf0.286Ta0.143Ti0.286Zr0.286
631,88,Hf0.294 Ta0.118 Ti0.294 Zr0.294,BCC+HCP,CAST,other,NaN,NaN,9.1,NaN,T,25.0,354.0,1100.0,30.6,NaN,NaN,99.0,NaN,NaN,NaN,10.1002/adma.201701678,2017.0,Phase-Transformation Ductilization of Brittle ...,10.1002.adma.201701678,Hf0.294Ta0.118Ti0.294Zr0.294
